# Import

In [69]:
import pandas as pd
import numpy as np
import statistics as stats
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns
from array import *
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


In [70]:
df = pd.read_csv('fifa21_train.csv')
df_val = pd.read_csv('fifa21_validate.csv')

# Function to clean the dataframes

In [71]:
def cleantable(df):
    # Standarize header
    df.drop(['ID'], axis=1, inplace=True)
    df.columns= df.columns.str.lower().str.replace(' ', '_')
    df.columns= df.columns.str.lower().str.replace('/', '_')
    df.columns= df.columns.str.lower().str.replace('&', 'and')
    same = df['lf'] == df['rf']
    same.value_counts()
    same = df['lf'] == df['cf']  
    same.value_counts()
    df.drop(['lf', 'rf'], axis=1, inplace=True)
    same = df['ls'] == df['st']  
    same.value_counts()
    same = df['ls'] == df['rs']  
    same.value_counts()
    df.drop(['ls', 'rs'], axis=1, inplace=True)
    same = df['lam'] == df['ram']  
    same.value_counts()
    same = df['lam'] == df['cam']  
    same.value_counts()
    df.drop(['lam', 'ram'], axis=1, inplace=True)
    same = df['lm'] == df['rm']  
    same.value_counts()
    df.drop(['rm'], axis=1, inplace=True)
    same = df['lcm'] == df['rcm']  
    same.value_counts()
    same = df['cm'] == df['rcm']  
    same.value_counts()
    df.drop(['rcm', 'lcm'], axis=1, inplace=True)
    same = df['ldm'] == df['rdm']  
    same.value_counts()
    same = df['ldm'] == df['cdm']  
    same.value_counts()
    df.drop(['rdm', 'ldm'], axis=1, inplace=True)
    same = df['lwb'] == df['rwb']  
    same.value_counts()
    df.drop(['rwb'], axis=1, inplace=True)
    same = df['lb'] == df['rb']  
    same.value_counts()
    df.drop(['rb'], axis=1, inplace=True)
    same = df['lcb'] == df['rcb']  
    same.value_counts()
    same = df['lcb'] == df['cb']  
    same.value_counts()
    df.drop(['lcb', 'rcb'], axis=1, inplace=True)
    same = df['cf'] == df['rw']  
    same.value_counts()
    same = df['lw'] == df['rw']  
    same.value_counts()
    df.drop(['rw'], axis=1, inplace=True)
    df['test'] = df['club'] + ' ' + df['contract']
    same = df['test'] == df['team_and_contract']
    same.value_counts()
    df.drop(['test'], axis=1, inplace=True)
    df.drop(['team_and_contract'], axis=1, inplace=True)
    #>90% of this column doesn't have a value
    df.drop(['loan_date_end'], axis=1, inplace=True)
    # Replace numerical values with the mean
    # mean & median have been quite similar 
    # round them, so they can be converted to int like the other values in this column
    na_cols = df.columns[df.isna().any()]
    for col in na_cols:        
        if df[col].dtypes == 'float64' or df[col].dtypes == 'int64':
            df[col].fillna(round(np.mean(df[col]),0), inplace=True)
    # Replace null categorical values with 'unknown'
    for col in na_cols:
        if df[col].dtypes == 'object':
            df[col].fillna('unknown', inplace=True)
    df['joined'] = pd.to_datetime(df['joined'], errors='coerce')
    df['joined'].fillna(stats.mode(df['joined']), inplace=True)        
    df['weight'] = df['weight'].str.replace('lbs','')
    # df['value'] =  df['value'].str.replace('K','')
    df['value'] =  df['value'].str.replace('€','')
    df['w_f'] =  df['w_f'].str.replace('★','')
    df['sm'] =  df['sm'].str.replace('★','')
    df['ir'] =  df['ir'].str.replace('★','')
    df['weight'] = df['weight'].astype(int)
    df['w_f'] = df['w_f'].astype(int)
    df['sm'] = df['sm'].astype(int)
    df['ir'] = df['ir'].astype(int)
    heightlist = []
    for row in df['height']:
        row = row.replace('"', '')
        split = row.split("'")
        height = int(split[0]) * 30.48 + int(split[1]) * 2.54
        heightlist.append(height)
    df['height'] = heightlist       
    df['height'] = df['height'].astype(int)
    for col in ['st', 'lw','cf', 'cam', 'lm', 'cm', 'lwb', 'cdm', 'lb', 'cb', 'gk']:
        colvalue = []
        coladdvalue = []
        for row in df[col]:
            split = row.split("+")
            colvalue.append(split[0])
            coladdvalue.append(split[1])
        df[col] = colvalue
        df[col + 'add'] = coladdvalue
        df[col] = df[col].astype(int)
        df[col + 'add'] = df[col + 'add'].astype(int)     
        df['st'] = df['st'].astype(int)   
        contract_to = []
    for row in df['contract']:
        split = row.split(" ~ ")  
        if len(split) == 2:
            contract_to.append(split[1])   
        else:
            contract_to.append('0')
    df['contract_to'] = contract_to       
    df['contract_to'] = df['contract_to'].astype(int)   
    # df['contract_to'] = pd.to_datetime(df['contract_to'], errors='coerce')
    #correcting values
    df['hits_K'] = df['hits'].astype(str).str[-1]
    df['hits_K'] = df['hits_K'].str.replace('0','1')
    df['hits_K'] = df['hits_K'].str.replace('K','100')
    df['hits_K'] = pd.to_numeric(df['hits_K'])
    df['hits'] = df['hits'].str.replace('K','')
    df['hits'] = pd.to_numeric(df['hits'])
    df['hits'] = round(df['hits'] * df['hits_K'],2)
    df.drop(['hits_K'], axis=1, inplace=True)
    #correcting values
    df['value_KoM'] = df['value'].astype(str).str[-1]
    df['value_KoM'] = df['value_KoM'].str.replace('0','1')
    df['value_KoM'] = df['value_KoM'].str.replace('M','100000')
    df['value_KoM'] = df['value_KoM'].str.replace('K','100')
    df['value_KoM'] = pd.to_numeric(df['value_KoM'])
    df['value'] = df['value'].str.replace('€','')
    df['value'] = df['value'].str.replace('M','')
    df['value'] = df['value'].str.replace('K','')
    df['value'] = pd.to_numeric(df['value'])
    df['value'] = round(df['value'] * df['value_KoM'],2)
    df.drop(['value_KoM'], axis=1, inplace=True)
    #correcting wages
    df['wage_KoM'] = df['wage'].astype(str).str[-1]
    df['wage_KoM'] = df['wage_KoM'].str.replace('0','1')
    df['wage_KoM'] = df['wage_KoM'].str.replace('M','1000000')
    df['wage_KoM'] = df['wage_KoM'].str.replace('K','1000')
    df['wage_KoM'] = pd.to_numeric(df['wage_KoM'])
    df['wage'] = df['wage'].str.replace('€','')
    df['wage'] = df['wage'].str.replace('M','')
    df['wage'] = df['wage'].str.replace('K','')
    df['wage'] = pd.to_numeric(df['wage'])
    df['wage'] = round(df['wage'] * df['wage_KoM'],2)
    df.drop(['wage_KoM'], axis=1, inplace=True) 
    #correcting release_clauses
    df['release_clause_KoM'] = df['release_clause'].astype(str).str[-1]
    df['release_clause_KoM'] = df['release_clause_KoM'].str.replace('0','1')
    df['release_clause_KoM'] = df['release_clause_KoM'].str.replace('M','100000')
    df['release_clause_KoM'] = df['release_clause_KoM'].str.replace('K','100')
    df['release_clause_KoM'] = pd.to_numeric(df['release_clause_KoM'])
    df['release_clause'] = df['release_clause'].str.replace('€','')
    df['release_clause'] = df['release_clause'].str.replace('M','')
    df['release_clause'] = df['release_clause'].str.replace('K','')
    df['release_clause'] = pd.to_numeric(df['release_clause'])
    df['release_clause'] = round(df['release_clause'] * df['release_clause_KoM'],2)
    df.drop(['release_clause_KoM'], axis=1, inplace=True)
    df = df[['name', 'age', 'nationality', 'club', 'bp', 'position', 'height',
       'weight', 'foot', 'growth', 'joined', 'value', 'wage', 'release_clause',
       'contract', 'contract_to', 'attacking', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'hits', 'st','stadd',  'lw','lwadd', 'cf','cfadd', 'cam','camadd', 'lm', 'lmadd', 'cm', 'lwb','lwbadd',
       'cmadd','cdm','cdmadd', 'lb', 'lbadd','cb','cbadd', 'gk', 'gkadd', 'ova' 
       ]]
    return df

# Run the function on both tables

In [ ]:
cleantable(df_val)
cleantable(df)

# Create the LR model

In [ ]:
## Training the model
y = df['ova']
X = df.drop(['ova'], axis=1)
X_num = X._get_numeric_data()
X_cat = X.select_dtypes('object')

# Normalizing data: make data range from 0 - 1, instead of from min to max
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
x_normalized #this is an array
X_num = pd.DataFrame(x_normalized, columns=X_num.columns) #now we convert the array to a dataframe

X_cat = X_cat.drop(['name'], axis=1)
X_cat = X_cat.drop(['position'], axis=1)
X_cat = X_cat.drop(['contract'], axis=1)
X_cat = X_cat.drop(['nationality'], axis=1)
X_cat = X_cat.drop(['club'], axis=1)

encoder = OneHotEncoder().fit(X_cat)
encoded_for_p = encoder.transform(X_cat).toarray()
encoded_for_p
encoder.categories_

cols=[colname for row in encoder.categories_ for colname in row]
cols
onehot_encoded_for_p = pd.DataFrame(encoded_for_p,columns=cols)
cols_to_drop=[row[0] for row in encoder.categories_]
cols_to_drop
onehot_encoded_for_p = onehot_encoded_for_p.drop(cols_to_drop,axis=1)
onehot_encoded_for_p.head()
# print(encoded_for_p)
# print(cols)

X = pd.concat([X_num, onehot_encoded_for_p], axis=1)
# X = X_num
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
predictions = lm.predict(X_test)
r2_score(y_test, predictions)



# Run the LR model against the validation set

In [ ]:
## Running the model against the validation set
y_val = df['ova']
X_val = df.drop(['ova'], axis=1)
X_val_num = X_val._get_numeric_data()
X_val_cat = X_val.select_dtypes('object')

# Normalizing data: make data range from 0 - 1, instead of from min to max
transformer = MinMaxScaler().fit(X_val_num)
x_normalized = transformer.transform(X_val_num)
print(x_normalized.shape)
x_normalized #this is an array
X_val_num = pd.DataFrame(x_normalized, columns=X_num.columns) #now we convert the array to a dataframe

X_val_cat = X_val_cat.drop(['name'], axis=1)
X_val_cat = X_val_cat.drop(['position'], axis=1)
X_val_cat = X_val_cat.drop(['contract'], axis=1)
X_val_cat = X_val_cat.drop(['nationality'], axis=1)
X_val_cat = X_val_cat.drop(['club'], axis=1)

encoder = OneHotEncoder().fit(X_val_cat)
encoded_for_p = encoder.transform(X_val_cat).toarray()
encoded_for_p
encoder.categories_

cols=[colname for row in encoder.categories_ for colname in row]
cols
onehot_encoded_for_p = pd.DataFrame(encoded_for_p,columns=cols)
cols_to_drop=[row[0] for row in encoder.categories_]
cols_to_drop
onehot_encoded_for_p = onehot_encoded_for_p.drop(cols_to_drop,axis=1)
onehot_encoded_for_p.head()
# print(encoded_for_p)
# print(cols)

X_val = pd.concat([X_val_num, onehot_encoded_for_p], axis=1)


In [ ]:
predictions = lm.predict(X_val)
r2_score(y_val, predictions)

